### Rules:

1- Any live cell with fewer than two live neighbours dies, as if by underpopulation.

2- Any live cell with two or three live neighbours lives on to the next generation.

3- Any live cell with more than three live neighbours dies, as if by overpopulation.

4- Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

In [1]:
import pygame # we use pygame
import numpy as np # numpy to repr the array
from time import sleep # to control the speed of the game 
#from numba import njit # to speed up everything

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


Basic settings:

In [5]:
width, height = 1100, 600
pixel = 5

# basic color
white = (255, 255, 255)
black = (0, 0, 0)

n = 12.7 * np.log(.02 * width*height/pixel**2 + 1.2) # to understand why look at the end of Stat notebook
n = round(n) # This is the number of color
print('The world is supposed to converge roughly in',n,'step')

palette = [(int(i*255/n),int(i*255/n),255) for i in range(n,-1,-1)]

The world is supposed to converge roughly in 80 step


We first create a world. (we will use pygame to show our results and provide an interactive enviroment for the user)

In [3]:
# function that takes in a world, and updates it using the first three rules above
#@njit("i1[:,:](i1[:,:])", nogil=True)
def next_gen(world,color):
    
    height, width = world.shape # take world shape
    new_world = np.zeros((height, width), np.int8) # make a new world that we want to edit
    
    for i in range(height): # iterate through all the rows
        for j in range(width):  # iterate through all columns
            
            cell = world[i][j]
            neighbours_count = np.array([
                    world[i_r][j_r]
                    for i_r in ((i-1)%height, i, (i+1)%height) #periodic condition applied
                    for j_r in ((j-1)%width, j, (j+1)%width)
            ]).sum() - cell
            
            if cell and neighbours_count in (2, 3):
                new_world[i][j] = 1
                if color[i][j] < n: color[i][j] += 1
                
            elif (not cell) and neighbours_count == 3:
                new_world[i][j] = 1
            
    return new_world,color

In [4]:
pygame.init()

screen = pygame.display.set_mode((width, height))

world = np.random.randint(0, 2, (height//pixel, width//pixel), np.int8)
#world = np.zeros((height//pixel, width//pixel), np.int8)
color = world.copy()

# now we want to take the initial position as input from the user
def user_edit(world):
    # if left click: make cell alive
    if pygame.mouse.get_pressed()[0]: # if left click is pressed
        x, y = pygame.mouse.get_pos()  # take the cursor position and try to highlight on grid
        world[y//pixel][x//pixel] = 1
        
    # if right click: make cell dead
    if pygame.mouse.get_pressed()[2]:
        x, y = pygame.mouse.get_pos()
        world[y//pixel][x//pixel] = 0
    
    return world

def update_screen(world,color):
    height, width = world.shape
    screen.fill(black)
    for i in range(height):
        for j in range(width):
            color_index = color[i][j] 
            if world[i][j]:
                pygame.draw.rect(screen, palette[color_index], (j*pixel, i*pixel, pixel, pixel))
            else:
                color[i][j] = 0
                
run = True
pause = True
time_to_sleep = 0.3

while run:

    update_screen(world,color)
    if not pause:
        sleep(time_to_sleep)
        world,color = next_gen(world,color)
            
    if pause:
        world = user_edit(world)
    
    pygame.display.update()
    
    for event in pygame.event.get():
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 2:       
                pause = not pause
                
            if event.button == 4:
                time_to_sleep = max(0, time_to_sleep + 0.05)
                time_to_sleep = min(time_to_sleep, 1)
                
            if event.button == 5:
                time_to_sleep = max(0, time_to_sleep - 0.05)
                time_to_sleep = min(time_to_sleep, 1)
                
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.QUIT
            run = False